In [1]:
## call the main routines
using NBInclude
@nbinclude("core_fun.ipynb");

We fix the parameters for the case of Germany in an object of type Country. The calibrated values for $\alpha$, $m$, $q$, $\lambda_{min}$ and $\lambda_{max}$ are obtained latter on.

In [2]:
germany = let 
    
    reform = 2
    A = 42. * 12
    Amin = 38. * 12
    Amax = 44. * 12
    
    r = 0.0048 # 5 % annual interest rate, time unit = month
    c0 = 1. # scale parameter of the cost function, normalized to 1

    ### to be updated after the calibration
    alpha, m, q = 0.7 , 0.1, 0.03
    lambdamin = 0.07
    lambdamax = lambdamin + 0.03
    
    uinit = [0.,0.] ## useless parameters
    
    wbar = 1.
    bUI = 0.68 * wbar  # replacement rate for individual with children 68 (without children is 63) , SVB2012a
    b0max = 0.35 * wbar # 35% social assistance for married individuals, SVB2012a
    b0min = b0max
    
    Country(reform,A,Amin,Amax,r,c0,alpha,q,m,lambdamin,lambdamax,uinit,bUI,b0max,b0min,wbar)
end
;

In [3]:
## precompile functions
let 
    a=41.8*12
    B=6.2
    D=15.4
    simu = find_simu(germany,reltol=1e-5) 
    @time fUI(B, a, simu,germany) 
    @time expectedB(a, simu,germany)
    @time integrand_fNE(B, D, a,simu, germany)
    @time fNE(D, a, simu, germany) 
    @time expectedD(a, simu, germany)
    @time averageU5(0.025, 0.07, 0.0537, germany) 
end

  2.618157 seconds (5.86 M allocations: 295.838 MiB, 5.97% gc time)
  0.402507 seconds (1.53 M allocations: 49.306 MiB, 5.65% gc time)
  0.571530 seconds (648.24 k allocations: 31.982 MiB, 1.76% gc time)
  0.482942 seconds (979.49 k allocations: 40.128 MiB, 4.63% gc time)
 11.578129 seconds (119.55 M allocations: 2.044 GiB, 4.77% gc time)
  7.725521 seconds (85.43 M allocations: 1.626 GiB, 5.77% gc time)


29.12427721560897

# 1) Find the parameters for different values of $q$
We take as moments the expected duration of nonemployment and UI.

In [4]:
function moments_germany(simu::Simu,C::Country)
    momD1 = expectedD(41.99*12,simu,C)
    momD2 = expectedD(42.01*12,simu,C)
  #  jfr0 = JFrate(simu.Ybar[1],C)
  #  jfrUI = JFrate(simu.Ybar[3],C)
  #  momav = averageU5(C.q, jfr0, jfrUI, C) 
    momB1 = expectedB(41.99*12,simu,C)
    momB2 = expectedB(42.01*12,simu,C)
    return([momD1, momD2, momB1, momB2])
end

moments_target = [15.6, 16.4, 6.7, 8.5] 

function distance(m1,m2)
    out = 0
    for i in 1:length(m1)
        out += (m1[i] - m2[i]) ^2
    end
    return(out)
end

distance (generic function with 1 method)

We calibrate our model to determine the values of $\alpha$, $m$, $\lambda_{min}$ and $\lambda_{max}$ for different values of $q$.

In [5]:
## generate a Country object with new values for alpha, m, q , lambdamin and lambdamin+diff
function update_Country(alpha,m,q,lambdamin,lambdadiff, C::Country)
    Country(C.reform,C.A,C.Amin,C.Amax,C.r,C.c0,
        alpha,q, m, lambdamin, lambdamin+lambdadiff,
        C.uinit,C.bUI,C.b0max,C.b0min,C.wbar)
end

## compute the objective to minimise
function objective_ger(alpha,m,q,lambdamin,lambdadiff, C::Country)
    newC = update_Country(alpha,m,q,lambdamin,lambdadiff, C)

    ## simulate
    simu = find_simu(newC,reltol=1e-5) 
    moments_emp = moments_germany(simu,newC)
    
    IJulia.clear_output(true)
    println(alpha," ",m," ",q)
    println(moments_emp)
    
    distance(moments_emp, moments_target)
end

function find_germany(q, C::Country)
    opt = Opt(:LN_BOBYQA, 4)
    lower_bounds!(opt, [0.1 , 0., 0., 0.])
    upper_bounds!(opt, [10. , 1., 0.2, 1.])
    xtol_rel!(opt,1e-8)
    stopval!(opt,1e-8)
    min_objective!(opt, (v,g)-> objective_ger(v[1],v[2],q,v[3],v[4],C))

    init = [0.7, 0.1, 0.06, 0.03]

    (minf,minx,ret) = optimize(opt, init) 
    println("Param found with convergence ", ret, ", the value of the objective is ",minf)
    println(minx[1], "  ",minx[2])
    return(minx)
end

q_vec = [0.005, 0.01, 0.025, 0.05, 0.1]
am_vec = zeros(4,5)
for (i,q) in enumerate(q_vec)
    am_vec[:,i] =  find_germany(q,germany)
end
am_vec 

1.863908137661734 0.13715267414523216 0.1
[15.6, 16.4, 6.69999, 8.50004]
Param found with convergence STOPVAL_REACHED, the value of the objective is 3.1642415093585704e-9
1.863908137661734  0.13715267414523216


4×5 Array{Float64,2}:
 0.361291   0.479444   0.77479    1.18203    1.86391  
 0.0618409  0.0787383  0.108496   0.128621   0.137153 
 0.0659611  0.0659002  0.0657449  0.0655348  0.0652047
 0.0283165  0.0282879  0.0282176  0.028129   0.0279901

# 2) Statistics for each value of $q$

In [6]:
jfrUI_bef_vec = zeros(length(q_vec))
jfrUI_lmax_aft_vec = zeros(length(q_vec))
jfrUI_lmin_aft_vec = zeros(length(q_vec))
mom_vec = zeros(length(moments_target),length(q_vec))

for (i,q) in enumerate(q_vec)
    newC = update_Country(am_vec[1,i], am_vec[2,i],q,am_vec[3,i], am_vec[4,i], germany)
    simu_ger = find_simu(newC,reltol=1e-5)
    jfrUI_bef_vec[i] = JFrate(simu_ger.Xbar2(40*12),newC)
    jfrUI_lmax_aft_vec[i] = JFrate(simu_ger.Ybar[2],newC)
    jfrUI_lmin_aft_vec[i] = JFrate(simu_ger.Ybar[3],newC)

    mom_vec[:,i] = moments_germany(simu_ger,newC)
end

mom_vec
mid_vec = vcat(jfrUI_bef_vec',jfrUI_lmax_aft_vec',jfrUI_lmin_aft_vec')

3×5 Array{Float64,2}:
 0.0547839  0.0546959  0.0544615  0.0541921  0.0539528
 0.0549767  0.0550668  0.0552903  0.0555901  0.0560596
 0.0516853  0.0517472  0.0519026  0.0521131  0.0524418

# 3) RDD for each value of $q$

In [7]:
coefUI_vec = zeros(length(q_vec))
coefNE_vec = zeros(length(q_vec))
lateUI_vec = zeros(length(q_vec))
lateNE_vec = zeros(length(q_vec))

## simulated data at dayly frequency 
band = 2
age_grid = (germany.A/12-band):(1/365):(germany.A/12+band)
for (i,q) in enumerate(q_vec)
    newC = update_Country(am_vec[1,i], am_vec[2,i],q,am_vec[3,i], am_vec[4,i], germany)
    simu_ger = find_simu(newC,reltol=1e-5)
    
    Y1 = [expectedB(m,simu_ger,newC) for m in 12*age_grid] 
    Y2 = [expectedD(m,simu_ger,newC) for m in 12*age_grid] 

    #explaining variables
    TREAT = [a>=germany.A for a in age_grid*12]
    consta = ones(size(age_grid))
    age1 = collect(age_grid*12)
    age1_spl = (age1 .- germany.A) .* TREAT 

    ## linear spline
    X = hcat(TREAT, consta, age1, age1_spl )   
    beta1 = inv(X' * X) * (X' * Y1)
    beta2 = inv(X' * X) * (X' * Y2)
    coefUI_vec[i] = beta1[1]  + beta1[4] * germany.A/1200 
    coefNE_vec[i] = beta2[1]  + beta2[4] * germany.A/1200 
    
    ## simulated effect of the policy at age A
    mu0 = JFrate(simu_ger.Ybar[4],newC)
    muUI = JFrate(simu_ger.Ybar[5],newC) 
    lam = newC.lambdamax
    expDcounter0 = ((mu0-muUI)/((lam+muUI)*(mu0-lam-muUI)) - lam/(mu0*(mu0-lam-muUI)))
    expDcounter1 = expectedD(germany.Amax,simu_ger,newC)

    expBcounter0 = 1/ (newC.lambdamax+JFrate(simu_ger.Ybar[5],newC))
    expBcounter1 = expectedB(germany.Amax,simu_ger,newC)

    lateUI_vec[i] = expBcounter1-expBcounter0
    lateNE_vec[i] = expDcounter1-expDcounter0

end

# The final table is

In [8]:
am_vec[4,:] = am_vec[4,:] + am_vec[3,:] ## obtain lambdamax
vcat(q_vec',am_vec,mid_vec,1.78 .* ones(length(q_vec))',coefUI_vec',lateUI_vec',0.78 .* ones(length(q_vec))',coefNE_vec',lateNE_vec')

14×5 Array{Float64,2}:
 0.005      0.01       0.025      0.05       0.1      
 0.361291   0.479444   0.77479    1.18203    1.86391  
 0.0618409  0.0787383  0.108496   0.128621   0.137153 
 0.0659611  0.0659002  0.0657449  0.0655348  0.0652047
 0.0942777  0.0941881  0.0939625  0.0936638  0.0931948
 0.0547839  0.0546959  0.0544615  0.0541921  0.0539528
 0.0549767  0.0550668  0.0552903  0.0555901  0.0560596
 0.0516853  0.0517472  0.0519026  0.0521131  0.0524418
 1.78       1.78       1.78       1.78       1.78     
 1.80069    1.80118    1.80216    1.8027     1.80115  
 1.78982    1.78071    1.75868    1.73338    1.70304  
 0.78       0.78       0.78       0.78       0.78     
 0.803662   0.807087   0.814728   0.821499   0.823033 
 0.74539    0.698783   0.591631   0.478924   0.364201 